In [13]:
import polars as pl
import polars.selectors as cs
import os

### Confirm Files

In [15]:
import glob

csv_files = glob.glob("data/lahman_1871-2024_csv/*.csv")

csv_files


['data/lahman_1871-2024_csv\\AllstarFull.csv',
 'data/lahman_1871-2024_csv\\Appearances.csv',
 'data/lahman_1871-2024_csv\\AwardsManagers.csv',
 'data/lahman_1871-2024_csv\\AwardsPlayers.csv',
 'data/lahman_1871-2024_csv\\AwardsShareManagers.csv',
 'data/lahman_1871-2024_csv\\AwardsSharePlayers.csv',
 'data/lahman_1871-2024_csv\\Batting.csv',
 'data/lahman_1871-2024_csv\\BattingPost.csv',
 'data/lahman_1871-2024_csv\\CollegePlaying.csv',
 'data/lahman_1871-2024_csv\\Fielding.csv',
 'data/lahman_1871-2024_csv\\FieldingOF.csv',
 'data/lahman_1871-2024_csv\\FieldingOFsplit.csv',
 'data/lahman_1871-2024_csv\\FieldingPost.csv',
 'data/lahman_1871-2024_csv\\HallOfFame.csv',
 'data/lahman_1871-2024_csv\\HomeGames.csv',
 'data/lahman_1871-2024_csv\\Managers.csv',
 'data/lahman_1871-2024_csv\\ManagersHalf.csv',
 'data/lahman_1871-2024_csv\\Parks.csv',
 'data/lahman_1871-2024_csv\\People.csv',
 'data/lahman_1871-2024_csv\\Pitching.csv',
 'data/lahman_1871-2024_csv\\PitchingPost.csv',
 'data/lahm

In [16]:

(salaries :=
 pl.read_csv('./data/lahman_1871-2024_csv/Salaries.csv')
      .filter(pl.col('yearID').is_between(2006, 2016))
)

salaries


yearID,teamID,lgID,playerID,salary
i64,str,str,str,i64
2007,"""CHA""","""AL""","""aardsda01""",387500
2008,"""BOS""","""AL""","""aardsda01""",403250
2009,"""SEA""","""AL""","""aardsda01""",419000
2010,"""SEA""","""AL""","""aardsda01""",2750000
2011,"""SEA""","""AL""","""aardsda01""",4500000
…,…,…,…,…
2010,"""DET""","""AL""","""zumayjo01""",915000
2011,"""DET""","""AL""","""zumayjo01""",1400000
2014,"""SEA""","""AL""","""zuninmi01""",504100


## Filtering to 2006-2016

In [17]:
pitching = (
    pl.read_csv('./data/lahman_1871-2024_csv/Pitching.csv')
      .filter(pl.col('yearID').is_between(2006, 2016))
      .group_by(["playerID", "yearID"])
      .agg([
          pl.col("W").sum(),
          pl.col("L").sum(),
          pl.col("G").sum(),
          pl.col("GS").sum(),
          pl.col("SV").sum(),
          pl.col("IPouts").sum(),
          pl.col("H").sum(),
          pl.col("ER").sum(),
          pl.col("HR").sum(),
          pl.col("BB").sum(),
          pl.col("SO").sum()
      ])
)
pitching

playerID,yearID,W,L,G,GS,SV,IPouts,H,ER,HR,BB,SO
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""weaveje02""",2016,12,12,31,31,0,534,209,100,37,51,103
"""mussimi01""",2008,20,9,34,34,0,601,214,75,17,31,150
"""galarar01""",2008,13,7,30,28,0,536,152,74,28,61,126
"""putzjj01""",2008,6,5,47,0,15,139,46,20,4,28,56
"""medlekr01""",2015,6,2,15,8,0,175,56,26,6,18,40
…,…,…,…,…,…,…,…,…,…,…,…,…
"""oflaher01""",2011,2,4,78,0,0,221,59,8,2,21,67
"""peacobr01""",2011,2,0,3,2,0,36,7,1,0,6,4
"""pettian01""",2009,14,8,32,32,0,584,193,90,20,76,148


In [18]:
(salaries :=
    pl.read_csv('./data/lahman_1871-2024_csv/Salaries.csv')
      .filter(pl.col('yearID').is_between(2006, 2016))
      .select(['playerID', 'yearID', 'salary'])
)

salaries

playerID,yearID,salary
str,i64,i64
"""aardsda01""",2007,387500
"""aardsda01""",2008,403250
"""aardsda01""",2009,419000
"""aardsda01""",2010,2750000
"""aardsda01""",2011,4500000
…,…,…
"""zumayjo01""",2010,915000
"""zumayjo01""",2011,1400000
"""zuninmi01""",2014,504100


In [19]:
(fielding_p :=
    pl.read_csv('./data/lahman_1871-2024_csv/Fielding.csv')
      .filter(pl.col('yearID').is_between(2006, 2016))
      .filter(pl.col('POS') == 'P')
      .group_by(['playerID', 'yearID'])
      .agg([
          pl.col('PO').sum(),
          pl.col('A').sum(),
          pl.col('E').sum(),
          pl.col('G').sum().alias('G_field')
      ])
)

fielding_p

playerID,yearID,PO,A,E,G_field
str,i64,i64,i64,i64,i64
"""morribr01""",2014,2,8,1,60
"""loewead01""",2016,0,0,0,8
"""bucknbi02""",2013,0,1,0,7
"""daviswa01""",2013,12,13,1,31
"""beavabl01""",2011,5,10,0,15
…,…,…,…,…,…
"""brachbr01""",2012,2,6,2,67
"""harderi01""",2006,6,7,0,9
"""tallebr01""",2011,3,3,0,19


### Feature Data

In [20]:
(features :=
    pitching
      .join(fielding_p, on=['playerID', 'yearID'], how='left')
)

features


playerID,yearID,W,L,G,GS,SV,IPouts,H,ER,HR,BB,SO,PO,A,E,G_field
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""weaveje02""",2016,12,12,31,31,0,534,209,100,37,51,103,6,7,1,31
"""mussimi01""",2008,20,9,34,34,0,601,214,75,17,31,150,18,23,1,34
"""galarar01""",2008,13,7,30,28,0,536,152,74,28,61,126,13,11,2,30
"""putzjj01""",2008,6,5,47,0,15,139,46,20,4,28,56,1,7,1,47
"""medlekr01""",2015,6,2,15,8,0,175,56,26,6,18,40,4,7,0,15
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""oflaher01""",2011,2,4,78,0,0,221,59,8,2,21,67,3,13,0,78
"""peacobr01""",2011,2,0,3,2,0,36,7,1,0,6,4,1,0,0,3
"""pettian01""",2009,14,8,32,32,0,584,193,90,20,76,148,3,27,2,32


### Final Training/Validation Table

In [21]:


(final_ds :=
    features
      .join(salaries, on=["playerID", "yearID"], how="left") 
      .with_columns(
          pl.when(pl.col("yearID") == 2016)
            .then(pl.lit("Validation"))
            .otherwise(pl.lit("Training"))
            .alias("TrainingValidation")
      )
      .sort(["yearID", "playerID"])
)

final_ds


playerID,yearID,W,L,G,GS,SV,IPouts,H,ER,HR,BB,SO,PO,A,E,G_field,salary,TrainingValidation
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2006,3,0,45,0,0,159,41,24,9,28,49,1,5,0,45,null,"""Training"""
"""abreuwi01""",2006,0,0,7,0,0,24,10,9,1,6,6,1,0,0,7,null,"""Training"""
"""accarje01""",2006,2,4,65,0,3,207,76,41,7,20,54,2,12,0,65,330000,"""Training"""
"""adamsmi03""",2006,0,0,2,0,0,7,4,3,1,2,1,0,0,0,2,null,"""Training"""
"""adkinjo01""",2006,2,1,55,0,0,163,55,24,3,20,30,1,12,1,55,null,"""Training"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""youngma03""",2016,0,0,8,0,0,21,12,5,0,4,4,0,2,0,8,null,"""Validation"""
"""zastrro01""",2016,1,0,8,1,0,48,12,2,0,5,17,1,2,0,8,null,"""Validation"""
"""zieglbr01""",2016,4,7,69,0,22,204,67,17,2,26,58,2,11,1,69,5500000,"""Validation"""


In [22]:
final_ds.group_by("TrainingValidation").agg(pl.len().alias("n"))


TrainingValidation,n
str,u32
"""Validation""",742
"""Training""",6681


### Table to CSV

In [23]:
final_ds.write_csv("final_ds.csv")